In [1]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
from config import (ServerName, UserName, Password, port, DataBase)

### Store CSV into DataFrame

In [2]:
csv_file = "../Resources/suicide.csv"
suicide_df = pd.read_csv(csv_file)
suicide_df.head()

,Unnamed: 0,Unnamed: 1,Crude suicide rates (per 100 000 population),Crude suicide rates (per 100 000 population).1,Crude suicide rates (per 100 000 population).2,Crude suicide rates (per 100 000 population).3,Crude suicide rates (per 100 000 population).4
0,Country,Sex,2016.0,2015.0,2010.0,2005.0,2000.0
1,Afghanistan,Both sexes,4.7,4.8,5.1,6.3,5.7
2,Afghanistan,Male,7.6,7.8,8.6,10.8,10.0
3,Afghanistan,Female,1.5,1.5,1.4,1.5,1.0
4,Albania,Both sexes,6.3,6.0,7.8,6.7,5.5


### Change header and drop first row

In [3]:
# Select first row as header
header = suicide_df.iloc[0] 
# Remove first row from the dataframe
suicide_df = suicide_df[1:]
#set the header row as the df header
suicide_df.columns = header
suicide_df

,Country,Sex,2016.0,2015.0,2010.0,2005.0,2000.0
1,Afghanistan,Both sexes,4.7,4.8,5.1,6.3,5.7
2,Afghanistan,Male,7.6,7.8,8.6,10.8,10.0
3,Afghanistan,Female,1.5,1.5,1.4,1.5,1.0
4,Albania,Both sexes,6.3,6.0,7.8,6.7,5.5
5,Albania,Male,7.9,7.6,9.5,7.7,7.4
...,...,...,...,...,...,...,...
545,Zambia,Male,8.8,8.7,9.0,9.3,11.8
546,Zambia,Female,3.5,3.5,3.6,3.8,4.6
547,Zimbabwe,Both sexes,10.7,10.6,11.9,13.9,12.9
548,Zimbabwe,Male,15.7,15.5,18.1,21.7,20.6


### Create new data with select columns

In [4]:
new_suicide_df = suicide_df[['Country', 'Sex', 2000, 2005, 2010, 2015, 2016]].copy()
new_suicide_df.head()

,Country,Sex,2000.0,2005.0,2010.0,2015.0,2016.0
1,Afghanistan,Both sexes,5.7,6.3,5.1,4.8,4.7
2,Afghanistan,Male,10.0,10.8,8.6,7.8,7.6
3,Afghanistan,Female,1.0,1.5,1.4,1.5,1.5
4,Albania,Both sexes,5.5,6.7,7.8,6.0,6.3
5,Albania,Male,7.4,7.7,9.5,7.6,7.9


### Rename Columns

In [5]:
new_suicide_df = new_suicide_df.rename(columns={2000.0: 2000, 2005.0: 2005, 2010.0: 2010, 
                                                2015.0: 2015, 2016.0: 2016})
new_suicide_df 

,Country,Sex,2000,2005,2010,2015,2016
1,Afghanistan,Both sexes,5.7,6.3,5.1,4.8,4.7
2,Afghanistan,Male,10.0,10.8,8.6,7.8,7.6
3,Afghanistan,Female,1.0,1.5,1.4,1.5,1.5
4,Albania,Both sexes,5.5,6.7,7.8,6.0,6.3
5,Albania,Male,7.4,7.7,9.5,7.6,7.9
...,...,...,...,...,...,...,...
545,Zambia,Male,11.8,9.3,9.0,8.7,8.8
546,Zambia,Female,4.6,3.8,3.6,3.5,3.5
547,Zimbabwe,Both sexes,12.9,13.9,11.9,10.6,10.7
548,Zimbabwe,Male,20.6,21.7,18.1,15.5,15.7


## Melt Years

In [6]:
new_suicide_df = pd.melt(new_suicide_df, id_vars=['Country', "Sex"], value_vars=[2000, 2005, 2010, 2015,2016])
new_suicide_df

,Country,Sex,0,value
0,Afghanistan,Both sexes,2000,5.7
1,Afghanistan,Male,2000,10.0
2,Afghanistan,Female,2000,1.0
3,Albania,Both sexes,2000,5.5
4,Albania,Male,2000,7.4
...,...,...,...,...
2740,Zambia,Male,2016,8.8
2741,Zambia,Female,2016,3.5
2742,Zimbabwe,Both sexes,2016,10.7
2743,Zimbabwe,Male,2016,15.7


## Rename Year Column

In [7]:
new_suicide_df = new_suicide_df.rename(columns={0: "Year", "value": "Crude_Suicide_Rate"})
new_suicide_df

,Country,Sex,Year,Crude_Suicide_Rate
0,Afghanistan,Both sexes,2000,5.7
1,Afghanistan,Male,2000,10.0
2,Afghanistan,Female,2000,1.0
3,Albania,Both sexes,2000,5.5
4,Albania,Male,2000,7.4
...,...,...,...,...
2740,Zambia,Male,2016,8.8
2741,Zambia,Female,2016,3.5
2742,Zimbabwe,Both sexes,2016,10.7
2743,Zimbabwe,Male,2016,15.7


### Replace Countries with equivalents

In [8]:
csv_file = "../Resources/countries_problems_equivalence.csv"
count_equiv = pd.read_csv(csv_file)
count_equiv = count_equiv.dropna()
count_equiv = count_equiv[count_equiv["Country"] != count_equiv["Equivalence"]]
count_equiv = count_equiv.set_index('Country')
dict = count_equiv.to_dict()
dict = dict["Equivalence"]
new_suicide_df = new_suicide_df.replace({"Country": dict})
new_suicide_df

,Country,Sex,Year,Crude_Suicide_Rate
0,Afghanistan,Both sexes,2000,5.7
1,Afghanistan,Male,2000,10.0
2,Afghanistan,Female,2000,1.0
3,Albania,Both sexes,2000,5.5
4,Albania,Male,2000,7.4
...,...,...,...,...
2740,Zambia,Male,2016,8.8
2741,Zambia,Female,2016,3.5
2742,Zimbabwe,Both sexes,2016,10.7
2743,Zimbabwe,Male,2016,15.7


### Reorganize Rows and Columns with Pivot Tables

In [9]:
new_suicide_df = pd.pivot_table(new_suicide_df, values='Crude_Suicide_Rate', index=['Country', 'Year'],
                     columns=['Sex'], aggfunc=np.sum)
new_suicide_df

Sex               Both sexes  Female  Male
Country     Year                          
Afghanistan 2000         5.7     1.0  10.0
            2005         6.3     1.5  10.8
            2010         5.1     1.4   8.6
            2015         4.8     1.5   7.8
            2016         4.7     1.5   7.6
...                      ...     ...   ...
Zimbabwe    2000        12.9     5.5  20.6
            2005        13.9     6.4  21.7
            2010        11.9     6.1  18.1
            2015        10.6     6.0  15.5
            2016        10.7     6.0  15.7

[910 rows x 3 columns]

### Rename Remaining Columns

In [10]:
new_suicide_df = new_suicide_df.rename(columns={"Female": "Suicide_Rate_Female", 
                                                  "Male": "Suicide_Rate_Male"})
new_suicide_df

Sex               Both sexes  Suicide_Rate_Female  Suicide_Rate_Male
Country     Year                                                    
Afghanistan 2000         5.7                  1.0               10.0
            2005         6.3                  1.5               10.8
            2010         5.1                  1.4                8.6
            2015         4.8                  1.5                7.8
            2016         4.7                  1.5                7.6
...                      ...                  ...                ...
Zimbabwe    2000        12.9                  5.5               20.6
            2005        13.9                  6.4               21.7
            2010        11.9                  6.1               18.1
            2015        10.6                  6.0               15.5
            2016        10.7                  6.0               15.7

[910 rows x 3 columns]

### Create new data with select columns

In [11]:
new_suicide_df = new_suicide_df[['Suicide_Rate_Female', 'Suicide_Rate_Male']].copy()
new_suicide_df

Sex               Suicide_Rate_Female  Suicide_Rate_Male
Country     Year                                        
Afghanistan 2000                  1.0               10.0
            2005                  1.5               10.8
            2010                  1.4                8.6
            2015                  1.5                7.8
            2016                  1.5                7.6
...                               ...                ...
Zimbabwe    2000                  5.5               20.6
            2005                  6.4               21.7
            2010                  6.1               18.1
            2015                  6.0               15.5
            2016                  6.0               15.7

[910 rows x 2 columns]

## Database Connection

In [12]:
rds_connection_string = f'{UserName}:{Password}@{ServerName}:{port}/{DataBase}'
engine = create_engine(f'postgresql://{rds_connection_string}')

### Consult tables in the Database

In [13]:
engine.table_names()

['Hum_Dev_Ind', 'Economic', 'Suicide']

## Save dataframes in database and query it

In [14]:
df = new_suicide_df
table_name = 'Suicide'

df.to_sql(name= table_name, con=engine, if_exists='replace', index=True)
query = pd.read_sql_query(f'select * from "{table_name}"', con=engine).head()
query

,Country,Year,Suicide_Rate_Female,Suicide_Rate_Male
0,Afghanistan,2000,1.0,10.0
1,Afghanistan,2005,1.5,10.8
2,Afghanistan,2010,1.4,8.6
3,Afghanistan,2015,1.5,7.8
4,Afghanistan,2016,1.5,7.6


In [15]:
# Result for the main notebook
print('All the suicide data was uploaded to the database:')
print(query)

All the suicide data was uploaded to the database:
       Country  Year  Suicide_Rate_Female  Suicide_Rate_Male
0  Afghanistan  2000                  1.0               10.0
1  Afghanistan  2005                  1.5               10.8
2  Afghanistan  2010                  1.4                8.6
3  Afghanistan  2015                  1.5                7.8
4  Afghanistan  2016                  1.5                7.6


In [16]:
string = ["Bolivia", "Bolivia (Plurinational State of)"]
df.loc[string] 

Sex           Suicide_Rate_Female  Suicide_Rate_Male
Country Year                                        
Bolivia 2000                 12.7               20.0
        2005                 11.4               19.2
        2010                 10.4               17.7
        2015                  8.8               15.9
        2016                  8.6               15.8